In [1]:
import hyperopt as hp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os

from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval 

import joblib


C:\Users\jonat\AppData\Local\Temp\ipykernel_19596\1297048602.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
cur_path  = os.getcwd()

df = pd.read_csv(cur_path + '/processed.csv')
pd.set_option('display.min_rows', 150)


In [3]:
df['label'] = df['label'].eq('Churn').mul(1)

X = df.drop("label", axis = 1)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# space = {
#     'n_estimators': hp.choice('n_estimators', range(50, 500)),
#     'max_depth': hp.choice('max_depth', [1, 5, 10, 20, 50, 75, 100, 150, 200]),
#     'min_samples_split': hp.choice('min_samples_split', [2, 3, 4, 5, 10, 20]),
#     'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 3, 4, 5]),
#     'bootstrap': hp.choice('bootstrap', [True, False]),
#     'criterion': hp.choice('criterion', ['gini', 'entropy']),
#     'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
# }

# kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

# def objective(params):
#     params['max_depth'] = int(params['max_depth'])
#     clf = RandomForestClassifier(**params)
#     print(params)
#     scores = cross_val_score(clf, X_train, y_train, cv = kfold, scoring = 'f1', n_jobs = -1)
#     print(scores)
#     best_score = mean(scores) 
#     loss = -best_score
#     return {'loss': loss, 'params': params, 'status': STATUS_OK}

# num_trials = Trials()
# best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 40, trials = num_trials)

randomForest = RandomForestClassifier(bootstrap= False, criterion= 'gini', max_depth = 75, max_features = 'sqrt', min_samples_leaf = 3, min_samples_split = 10, n_estimators = 159)
randomForest.fit(X_train, y_train)

y_pred = randomForest.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)

print('Accuracy', accuracy)
print('F1 Score', f1_score)
print('Classification Report:\n', classification_report(y_test, y_pred))

Accuracy 0.951369293373333
F1 Score 0.9694512988678928
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.89      0.88     27180
           1       0.97      0.97      0.97    107488

    accuracy                           0.95    134668
   macro avg       0.92      0.93      0.93    134668
weighted avg       0.95      0.95      0.95    134668



In [ ]:
joblib.dump(randomForest, 'randomForest.pkl')